In [45]:
import numpy as np
import pandas as pd
from osgeo import gdal
from osgeo import osr
import os
import time
import math
import folium
from folium.plugins import HeatMap
from scipy.special import *
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

In [46]:
# def expn(x):
#     t = [-0.9061798459, -0.5384693101, 0.0, 0.5384693101, 0.9061798459]
#     c = [0.2369268851, 0.4786286705, 0.5688888889, 0.4786286705, 0.2369268851]
#     m = 1
#     r = 0.57721566490153286060651
#     if (x == 0):
#         x = 1.0e-10
#     if (x < 0.0):
#         x = -x
#     q = r + log(x)
#     h = x
#     s = fabs(0.0001*h)
#     p = 1.0e+35
#     ep = 0.000001
#     g = 0.0
#     while(ep >= 0.0000001) and (fabs(h) > s):
#         g = 0.0
#         for i in range(1,m+1):
#             aa = (i-1.0)*h 
#             bb = i*h
#             w = 0.0
#             for j in range(5):
#                 xx = ((bb - aa) * t[j] + (bb + aa)) /2.0
#                 w = w + (exp(-xx) - 1.0) / xx * c[j]
#             g = g + w
#         g = g * h / 2.0
#         ep = fabs(g-p) / (1.0 + fabs(g))
#         p = g
#         m = m + 1
#         h = x / m
#     g = q + g
#     return -g


In [47]:
# kGLAI=np.arange(1,10,0.01)
# integral1=np.exp(-kGLAI)/2*np.log(1+2/kGLAI)
# integral2=np.exp(-kGLAI)*np.log(1+1/kGLAI)
# integral=(integral1+integral2)/2
# plt.plot(kGLAI,integral)

In [48]:
# plt.plot(kGLAI,expn(1, kGLAI))

In [49]:
# plt.plot(kGLAI,expn(1, kGLAI)-integral)

In [50]:
def SZA_kb_calculation(Lat, Lon,doy):
    
    timeSeries=np.array([22.5,1.5,4.5,7.5,10.5,13.5,16.5,19.5])#三小时一个数据
    # timeSeries=np.array([12])#三小时一个数据
    # timeSeries=np.arange(1.5,24,3)#一小时一个数据
    # time_zone=np.where(Lon>0,np.int((Lon+7.5)/15),np.ceil((Lon+7.5)/15))
    time_zone=np.ceil((Lon+7.5)/15)-1
    local=(timeSeries+time_zone)%24
    LocalTime=np.where(local>=0,local,local+24)
    HourAngle = np.tile(15 * (LocalTime - 12),int(doy.shape[0]/timeSeries.shape[0]))
    Dec = -23.45 * np.cos(2 * np.pi * (doy + 10) / 365)
    cosSZA = np.sin(Lat * np.pi / 180) * np.sin(Dec * np.pi / 180) + np.cos(Lat * np.pi / 180) * np.cos(Dec * np.pi / 180) * np.cos(HourAngle * np.pi / 180)

    SZA = np.arccos(cosSZA) * 180 / np.pi  # Solar Zenith Angle
    SZA=np.where(SZA < 0,0,SZA)
    SZA=np.where(SZA >88,88,SZA)
    
    miukb = np.cos(SZA * np.pi / 180)
    Kai = 0.0  # Empirical parameter related to the leaf angle distribution, varying from -1 to 1 (1 for horizontal leaves, -1 for vertical leaves, and 0 for a spherical leaf angle distribution)
    Phi1 = 0.5 - 0.633 * Kai - 0.33 * (Kai ** 2)
    Phi2 = 0.877 * (1 - 2 * Phi1)
    G = Phi1 + Phi2 * miukb
    kb = G / miukb
    return SZA, miukb, G, kb ,HourAngle,Dec,cosSZA,LocalTime

In [51]:
def rad_transfer(LAI,kb,G):
    # Calculating radiation absorbed by canopy considering scattering in a finite canopy
    # coef_scattering = 0.15  # Mahat & Tabroton 2012: 0.5; CLM: 0.15; Wang & Leuning 1998: 0.2
    coef_scattering_b = 0.1
    coef_scattering_d = 0.65
    # k_p = math.sqrt(1 - coef_scattering)
    k_p_b = np.sqrt(1 - coef_scattering_b)
    k_p_d = np.sqrt(1 - coef_scattering_d)

    taob_p = np.exp(-k_p_b * kb * LAI)
    kGLAI=k_p_d * G * LAI
    # np.savetxt("C:\\Users\\DELL\\Downloads\\PSN_kGLAI.txt",kGLAI,fmt="%f",delimiter=" ")
    taod_p = (1 - kGLAI) * np.exp(-kGLAI) + (kGLAI) ** 2 *expn(1, kGLAI)
    # beta_p = (1 - k_p) / (1 + k_p)
    beta_p_b = (1 - k_p_b) / (1 + k_p_b)
    beta_p_d = (1 - k_p_d) / (1 + k_p_d)

    # For direct radiation and PAR
    taob = (taob_p * (1 - beta_p_b ** 2)) / (1 - beta_p_b ** 2 * taob_p ** 2)
    betab = (beta_p_b * (1 - taob_p ** 2)) / (1 - beta_p_b ** 2 * taob_p ** 2)

    # For diffuse radiation and PAR
    taod = (taod_p * (1 - beta_p_d ** 2)) / (1 - beta_p_d ** 2 * taod_p ** 2)
    betad = (beta_p_d * (1 - taod_p ** 2)) / (1 - beta_p_d ** 2 * taod_p ** 2)
    #tao_beer = math.exp(-kb * LAI)

    # Transmission factors for direct and diffuse radiation: taob and taod;
    # Reflection factors for direct and diffuse radiation: betab and betad.
    return taob, betab, taod, betad

In [52]:
def rad_partition(Rad,miukb):
    # Consider direct and diffuse radiation separately.
    # Partitioning to direct and diffuse radiation
    S0 = 1367 * miukb  # Solar constant W.m-2
    AT = Rad / S0
    As = 0.25  # As: fraction of extraterrestrial radiation S0 on overcast days.
    Bs = 0.50  # As + Bs: fraction of extraterrestrial radiation S0 on clear days.
    AT=np.where(AT > 0.75,0.75,AT)
    AT=np.where(AT < 0.25,0.25,AT)

    Cf = 1 - ((AT - As) / Bs)  # Cf: cloudiness fraction, 0 for clear sky and 1 for completely cloudy sky.
    #print(Cf)
    ATc = np.maximum(AT, As + Bs)
    lam = 6.0 / 7.0  # lam: when sky is clear that a fraction lam of AT is direct.
    ATb = lam * ATc * (1 - Cf)  # ATb: direct radiation fraction.

    Rad_direct=np.where(Rad == 0,0,(ATb / AT) * Rad)
    Rad_diffuse=np.where(Rad == 0,0,Rad - Rad_direct)
    return Rad_direct, Rad_diffuse  #PAR_direct, PAR_diffuse

In [53]:
def twoleaf(LAI, Rad_direct, Rad_diffuse, taob, betab, taod, betad, kb):
    kb_diffuse = - np.log(taod) / LAI   # log(x) returns the natural logarithm of x (to base e).
    #print(kb, kb_diffuse)
    ##Calculate the ratio of PAR to radiation for direct and diffuse seperately. ES lectures 3-2 P23: PAR = 0.42Sb + 0.6Sd
    coef_Rad2PAR_b = 0.42 #0.43
    coef_Rad2PAR_d = 0.6  #0.57
    # For the absorbed diffuse radiation:
    ARad_diffuse = Rad_diffuse * (1 - taod - betad)   # To the total canopy LAI
    # For the absorbed scattered part of direct radiation:
    Rad_direct_scattered = Rad_direct * taob / (np.exp(-kb_diffuse * LAI))
    ARad_direct_scattered = Rad_direct_scattered - Rad_direct * (taob + betab)
    ARad_direct_direct = Rad_direct * kb
    # Calculate LAI for sun and shade leaf, TWO-LEAF model
    LAI_sun = ((1 / kb) * (1 - np.exp(-kb * LAI)))  # Leaf area index of sun leaf (Projected) #LAI is projected LAI.
    LAI_shade = LAI - LAI_sun  # Leaf area index of shade leaf (Projected)
    # Absorbed PPFD by sunlit and shade leaves, which is used as inputs in Farquhar's PSN model
    APAR_direct_direct = ARad_direct_direct * coef_Rad2PAR_b * 4.57  # Covert W.m-2 to umol.m-2.s-1, umol.m-2.s-1
    APAR_direct_scattered = ARad_direct_scattered * coef_Rad2PAR_d * 4.57  # Covert W.m-2 to umol.m-2.s-1, umol.m-2.s-1
    APAR_diffuse = ARad_diffuse * coef_Rad2PAR_d * 4.57  # Covert W.m-2 to umol.m-2.s-1, umol.m-2.s-1

    PPFD_sun = (APAR_diffuse / LAI * LAI_sun + APAR_direct_scattered / LAI * LAI_sun + APAR_direct_direct) / LAI_sun
    PPFD_shade = (APAR_diffuse / LAI * LAI_shade + APAR_direct_scattered / LAI * LAI_shade) / LAI_shade
    #print(kb, kb_diffuse, LAI_sun, LAI_shade, PPFD_sun, PPFD_shade, APAR_diffuse, APAR_direct_scattered, PAR_direct_scattered, PAR_direct, taob, betab)

    return LAI_sun, LAI_shade, PPFD_sun, PPFD_shade

In [54]:
def PSN_C3(VPD, PPFD, Vcmax, Jmax, Tleaf, RH, gsmodel):
    ## define constant parameters:
    Ca = 400.0  # umol.mol-1
    Oi = 210.0  # partial pressure of O2 (mmol/mol)
    Patm = 101.0  # KPa
    g1 = 7.5  # Dimensionless. (Slope)  #Collatz et al.(1991)/Dai et al.(2004): g1=9,g0=0.01mol.m-2.s-1; Xu's dissertation (Ch7) and Panek and Goldstein (2000): g1=7.5,g0=0.01mol.m-2.s-1
    g0 = 0.01  # mol.m-2.s-1
    D0 = 2.0  # KPa          #Xu's dissertation (Ch7) and Panek and Goldstein (2000): D0=2000Pa
    # In BGC4.2 model,alpha=0.85/2.6 for C3 and alpha=0.85/3.5 for C4
    alpha = 0.85 / 2.6  # quantum yield of electron transport (mol electrons mol-1 photon)
    # In BGC4.2 model,theta = 0.7
    theta = 0.7  # curvature of the light response curve(dimensionless)
    Rd0 = 0.92  # umol.m-2.s-1(per leaf area) (dark respiration at 25 degree)
    Q10 = 2.0  # Dimensionless
    TrefR = 25.0  # DegreeC
    Q10_Kc = 2.1  # Dimensionless
    Q10_Ko = 1.2  # Dimensionless
    GCtoGW = 1.57  # conversion factor from  GC (Stamatal conductance for CO2) to GW (Stamatal conductance for H2O)

    ## Temperature-dependence of Kc, Ko and GammaStar, parameters from Medlyn et al. 2001.
    # (Temperature response of parameters of a biochemically based model of photosynthesis. II. A review of experimental data)
    # GammaStar is the CO2 compensation point in the absence of photorespiration (umol.mol-1), the unit of Tleaf should be degree C. (Eq.12 in Medlyn et al. 2001)
    GammaStar = 42.75 * np.exp(37830 * ((Tleaf + 273.15) - 298) / (298 * Rgas * (Tleaf + 273.15)))

    ## Rd is the rate of dark respiration (maintenance respiration)
    Rd = Rd0 * Q10 ** ((Tleaf - TrefR) / 10.0)

    ## correct kinetic constants for temperature, and do unit conversions
    # Kc is the michaelis constant for CO2 (umol.mol-1)
    # Ko is the michaelis constant for O2 (mmol.mol-1)
    # Ref.: Biome-BGC4.2
 
    Kc=np.where(Tleaf > 15,404.0 * (Q10_Kc ** ((Tleaf - 25.0) / 10.0)),\
               (404.0 * 1.8 * Q10_Kc ** ((Tleaf - 15.0) / 10.0)) / Q10_Kc)
    Ko = 248.0 * (Q10_Ko ** ((Tleaf - 25.0) / 10.0))
    Km = Kc * (1.0 + Oi / Ko)

    ## Get the solution of J from the equation: theta * J^2 - (alpha * Q + Jmax) * J + alpha * Q * Jmax = 0 (Medlyn et al. 2001)
    # J is the rate of electron transport, Q/PPFD is the incident photosynthetically active photon flux density.
    J = (alpha * PPFD + Jmax - np.sqrt((alpha * PPFD + Jmax) ** 2 - 4 * alpha * theta * PPFD * Jmax)) / (2 * theta)
    VJ = J / 4.0

    ## Different stomatal conductance model
    # "Ball Berry Leuning" model
    if (gsmodel == "BBL"):
        GSDIVA = g1 / (Ca * (1 + VPD / D0))
        GSDIVA = GSDIVA / GCtoGW
        # print(g1, Ca, VPD, D0, GSDIVA)
    # "Ball Berry" model
    if (gsmodel == "BB"):
        RH = RH / 100
        GSDIVA = g1 * RH / Ca
        GSDIVA = GSDIVA / GCtoGW


    ## Get CiC from Equation(a) & Equation(c) & Equation(d)
    a1 = g0 + GSDIVA * (Vcmax - Rd)
    b1 = (1 - Ca * GSDIVA) * (Vcmax - Rd) + g0 * (Km - Ca) - GSDIVA * (Vcmax * GammaStar + Km * Rd)
    c1 = -(1 - Ca * GSDIVA) * (Vcmax * GammaStar + Km * Rd) - g0 * Km * Ca
    # print(g0,GSDIVA,Vcmax,Rd)
    CIC = (-b1 + np.sqrt(b1 * b1 - 4 * a1 * c1)) / (2 * a1)
    ## Get CiJ from Equation(b) & Equation(c) & Equation(d)
    a2 = g0 + GSDIVA * (VJ - Rd)
    b2 = (1 - Ca * GSDIVA) * (VJ - Rd) + g0 * (2 * GammaStar - Ca) - GSDIVA * (VJ * GammaStar + 2 * GammaStar * Rd)
    c2 = -(1 - Ca * GSDIVA) * GammaStar * (VJ + 2 * Rd) - g0 * 2 * GammaStar * Ca
    #print(PPFD, J, VJ, a2, b2, c2, b2 ** 2 - 4 * a2 * c2)
    CIJ=np.where(a2==0,-c2 / b2,(-b2 + np.sqrt(b2 ** 2 - 4 * a2 * c2)) / (2 * a2))
    
    CIJ=np.where(PPFD == 0,Ca,CIJ)
    CIC=np.where(PPFD == 0,Ca,CIC)
        
    Ac = Vcmax * (CIC - GammaStar) / (CIC + Km)
    Aj = VJ * (CIJ - GammaStar) / (CIJ + 2 * GammaStar)

    ## Using min(Ac,Aj) to calculate stomatal conductance(GS)
    A = np.minimum(Ac, Aj)
    GS = g0 + GSDIVA * (A - Rd)

    ## give air temperature threshold (5 DegreeC) for phenology

    A=np.where(Tleaf > 5,A,0)
    return [A, Ac, Aj, GS, Rd]

def write_data(filepath, output_data):
    f = open(filepath, 'a')
    for line in output_data:
        for val in line:
            f.write(str(val) + ' ')
        f.write('\n')
    f.close()

In [55]:
# dir_path = r"D:\LAI_fitting_param\year_365"
# data=Dataset(os.path.join(dir_path, "lai_2007series.tif")).get_data( ) 
# Rad=Dataset(os.path.join(dir_path, "Rad_year_365.tif")).get_data( ) 
# SH=Dataset(os.path.join(dir_path, "SH_year_365.tif")).get_data( ) 
# temp_data=Dataset(os.path.join(dir_path, "Temp_year_365.tif"))
# Temp=temp_data.get_data( ) 
# Pressure=Dataset(os.path.join(dir_path, "Pressure_year_365.tif")).get_data( ) 

In [56]:
data=pd.read_csv(r"C:\Users\DELL\Downloads\PSN_shanhua118_real.txt",sep=' ')
output_path=r"C:\Users\DELL\Downloads\PSN_wangsj118_real.txt"
# LAI=2.0
Lon=data.iloc[0,0]
Lat=data.iloc[0,1]
Temp=np.array(data["Temp"])
Rad=np.array(data["Rad"])
SH=np.array(data["SH"])
Pressure=np.array(data["Pressure"])
LAI=np.array(data["LAI"])
# doy=data["doy"]


In [57]:
Rgas = 8.314  # ideal gas constant(J.K-1.mol-1)
# Vcmax25 = 43.50 #umol.m-2.s-1
Vcmax25 = 62.5  # umol.m-2.s-1  (per leaf area)(CLM4.5)
Jmax25 = 95.49  # umol.m-2.s-1  (per leaf area)
# Parameters for T-dependence of Vcmax and Jmax(peaked function,1942)
EaV = 65330  # J.mol-1  (CLM4.5)
EaJ = 43540  # J.mol-1  (CLM4.5)
# Choose the stomatal conductance model. "BB": "Ball Berry model"; "BBL": "Ball Berry Leuning model".
gsmodel = "BBL"

start=time.time()

Temp_point = Temp#[i,2::] # Convert K to C, degreeC
Rad_point = Rad#[i,2::]    # W.m-2
SH_point = SH#[i,2::]  # kg/kg
Pressure_point = Pressure#[i,2::]  # Pa

es = 0.61078 * np.exp((17.27 * Temp_point) / (Temp_point + 237.3))  # KPa
ea = 1.6077 * SH_point * Pressure_point * 0.001  # KPa
VPD = es - ea  # VPD:KPa
VPD=np.where(VPD<0,0,VPD)

RH = (ea / es) * 100  # RH:%
RH=np.where(RH<0,0,RH)
RH=np.where(RH>88,88,RH)

Tleaf = Temp_point
#     doy=np.arange(0,31,3/24)
doy=np.arange(1,366,1)
doy=np.repeat(doy,8)
SZA, miukb, G, kb ,HourAngle,Dec,cosSZA,LocalTime= SZA_kb_calculation(Lat, Lon,doy)
# print(SZA.shape)
# LAI = (data[i,2] + data[i,3] / (1 + np.exp(-data[i,4] * (doy - data[i,5]))) - data[i,3] / (1 + np.exp(-data[i,6] * (doy - data[i,7]))))*0.1
# LAI=5.2
taob, betab, taod, betad = rad_transfer(LAI,kb,G)
Rad_direct, Rad_diffuse = rad_partition(Rad_point,miukb)
LAI_sun, LAI_shade, PPFD_sun, PPFD_shade = twoleaf(LAI, Rad_direct, Rad_diffuse, taob, betab, taod, betad, kb)

## Calculate Vcmax based on Vcmax25 and leaf temperature
T_Vcmax = np.exp(EaV * ((Tleaf + 273.15) - 298) / (298 * Rgas * (Tleaf + 273.15)))
Vcmax = Vcmax25 * T_Vcmax
## Calculate Jmax for Aj based on Jmax25 and leaf temperature
T_Jmax = np.exp(EaJ * ((Tleaf + 273.15) - 298) / (298 * Rgas * (Tleaf + 273.15)))
Jmax = Jmax25 * T_Jmax
# Jmax = 2.1 * Vcmax

Vcmax_sun = Vcmax
Vcmax_shade = Vcmax
Jmax_sun = Jmax
Jmax_shade = Jmax
A_sun = PSN_C3(VPD, PPFD_sun, Vcmax_sun, Jmax_sun, Tleaf, RH, gsmodel)
A_shade = PSN_C3(VPD, PPFD_shade, Vcmax_shade, Jmax_shade, Tleaf, RH, gsmodel)

A_total = A_sun[0] * LAI_sun + A_shade[0] * LAI_shade
GS_total = A_sun[3] * LAI_sun + A_shade[3] * LAI_shade
Rd_total = A_sun[4] * LAI_sun + A_shade[4] * LAI_shade

result = []
output_header = ["Lon","Lat","N","LAI", "Temp","Rad","SH","Pressure",\
                 "SZA", "miukb", "G", "kb","HourAngle","Dec","cosSZA","LocalTime",\
                 "taob", "betab", "taod", "betad",\
                 "Rad_direct", "Rad_diffuse",\
                 "LAI_sun", "LAI_shade", "PPFD_sun", "PPFD_shade",\
                 "A_sun","A_shade",\
                 "PSN"]
result.append(output_header)
for i in range(A_total.size):
    tmp = []
    tmp.append(Lon)
    tmp.append(Lat)
    tmp.append(doy[i])
    tmp.append(LAI[i])

    tmp.append(Temp[i])
    tmp.append(Rad[i])
    tmp.append(SH[i])
    tmp.append(Pressure[i])
    
    tmp.append(SZA[i])
    tmp.append(miukb[i])
    tmp.append(G[i])
    tmp.append(kb[i])
    tmp.append(HourAngle[i])
    tmp.append(Dec[i])
    tmp.append(cosSZA[i])
    tmp.append(LocalTime[i%8])    

    tmp.append(taob[i])
    tmp.append(betab[i])
    tmp.append(taod[i])
    tmp.append(betad[i])

    tmp.append(Rad_direct[i])
    tmp.append(Rad_diffuse[i])

    tmp.append(LAI_sun[i])
    tmp.append(LAI_shade[i])
    tmp.append(PPFD_sun[i])
    tmp.append(PPFD_shade[i])

    tmp.append(A_sun[0][i])
    tmp.append(A_shade[0][i])
    tmp.append(A_total[i])

    result.append(tmp)
    
write_data(output_path, result)

print("total time:",time.time()-start)
    

total time: 0.08866453170776367


In [58]:

# start=time.time()

# Inform=temp_data.get_Trans_Proj()
# # [gtf, pro,xSize,ySize,Bands]
# output_raster = gdal.GetDriverByName('GTiff').Create('D:\Desktop\GPP_py01.tif',Inform[2], Inform[3], Inform[4] ,gdal.GDT_Float32)
# output_raster.SetGeoTransform(Inform[0])
# srs = osr.SpatialReference()
# srs.ImportFromEPSG(4326)

# output_raster.SetProjection(srs.ExportToWkt())

# for k in range(Inform[4]):
#     GPP=np.array(result_A)[:,k].reshape(Inform[3],Inform[2])
#     output_raster.GetRasterBand(k+1).WriteArray(GPP)
#     output_raster.GetRasterBand(k+1).SetDescription("p%s"%k)
# print("total time:",time.time()-start)    

In [59]:
# start=time.time()

# Inform=temp_data.get_Trans_Proj()
# # [gtf, pro,xSize,ySize,Bands]
# output_raster = gdal.GetDriverByName('GTiff').Create('D:\Desktop\GS01.tif',Inform[2], Inform[3], Inform[4] ,gdal.GDT_Float32)
# output_raster.SetGeoTransform(Inform[0])
# srs = osr.SpatialReference()
# srs.ImportFromEPSG(4326)

# output_raster.SetProjection(srs.ExportToWkt())

# for k in range(Inform[4]):
#     GS=np.array(result_GS)[:,k].reshape(Inform[3],Inform[2])
#     output_raster.GetRasterBand(k+1).WriteArray(GS)
#     output_raster.GetRasterBand(k+1).SetDescription("p%s"%k)
# print("total time:",time.time()-start)   

In [60]:
# start=time.time()

# Inform=temp_data.get_Trans_Proj()
# # [gtf, pro,xSize,ySize,Bands]
# output_raster = gdal.GetDriverByName('GTiff').Create('D:\Desktop\Rd01.tif',Inform[2], Inform[3], Inform[4] ,gdal.GDT_Float32)
# output_raster.SetGeoTransform(Inform[0])
# srs = osr.SpatialReference()
# srs.ImportFromEPSG(4326)

# output_raster.SetProjection(srs.ExportToWkt())

# for k in range(Inform[4]):
#     Rd=np.array(result_Rd)[:,k].reshape(Inform[3],Inform[2])
#     output_raster.GetRasterBand(k+1).WriteArray(Rd)
#     output_raster.GetRasterBand(k+1).SetDescription("p%s"%k)
# print("total time:",time.time()-start)  